In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Creating a database engine 

In [2]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///Chinook.sqlite")

In [3]:
table_names = engine.table_names()
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Querying 

In [4]:
con = engine.connect()
rs = con.execute("SELECT * from Employee")
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

In [5]:
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


### Opening an engine using context manager 

In [6]:
with engine.connect() as con:
    rs = con.execute("SELECT City, Country FROM Employee")
    df = pd.DataFrame(rs.fetchmany(size=5))
    df.columns = rs.keys()

df.head()

,City,Country
0,Edmonton,Canada
1,Calgary,Canada
2,Calgary,Canada
3,Calgary,Canada
4,Calgary,Canada


Using a context manager ensures that we dont have to manually close the connection. Also, rs.fetchmany has the argument size by which we can specify the number of rows required.

### Querying RDB directly with pandas 

In [8]:
df = pd.read_sql_query("SELECT * FROM Customer WHERE Country == 'Canada'", engine)
df.head(3)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
1,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,+1 (780) 434-5565,mphilips12@shaw.ca,5
2,15,Jennifer,Peterson,Rogers Canada,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,+1 (604) 688-2255,+1 (604) 688-8756,jenniferp@rogers.ca,3


### using table relationships 

In [9]:
df = pd.read_sql_query("SELECT * FROM Album ", engine)
df.head(3)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2


In [10]:
df = pd.read_sql_query("SELECT * FROM Artist ", engine)
df.head(3)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith


In [12]:
df = pd.read_sql_query(
    "SELECT Artist.Name, Album.Title FROM Artist INNER JOIN Album on Artist.ArtistId = Album.ArtistId ORDER BY Artist.Name DESC",
    engine,
)
df.head()

,Name,Title
0,Zeca Pagodinho,Ao Vivo [IMPORT]
1,Yo-Yo Ma,Bach: The Cello Suites
2,Yehudi Menuhin,Bartok: Violin & Viola Concertos
3,Wilhelm Kempff,Bach: Goldberg Variations
4,Vinícius De Moraes,Vinicius De Moraes


The query above joins the Name column from the Artist table and the Title column from the Album column using the common ArtistId attribute.